# optimization with caching

In [1]:
%pylab inline
import pandas as pd
import scipy.optimize
from scipy import stats
import multiprocessing
import sqlite3

Populating the interactive namespace from numpy and matplotlib


### Fit to one curve at a time
Fit the model to the first curve, saving the result of each optimization in the database
Before fitting to the second curve, find the place in the database with the lowest error to the second curve and begin the optimization there, repeat.

### Fit to multiple curves simultaneously
If multiple processes are executing the model, one may find a fit that is better than anything seen before, we should be able to jump to that...

If we can use the distributed system to begin the optimization, we can run them serially...


### hybrid
Look for the closest in the database, initialize there, run 3-5 steps in the integration, then stop and reinitialize.
Start the first optimization for a new curve at a random point in the space.

maintain a local database, farm out executions of the model to the nodes, which return the simulation result. Save the results locally

In [2]:
!rm model_runs.db

conn = sqlite3.connect('model_runs.db')

In [18]:
results_df = pd.read_sql('select * from %s;'%modelname, conn, index_col=['B0','B1'])
results_df = results_df[~results_df.index.duplicated(keep='first')] #drop duplicate indices
results_df

,,0,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
B0,B1,,,,,,,,,,,,,,,,,,,,,
1,2,0,1.098612,1.609438,1.94591,2.197225,2.397895,2.564949,2.70805,2.833213,2.944439,...,5.802118,5.808142,5.814131,5.820083,5.826,5.831882,5.83773,5.843544,5.849325,5.855072


In [20]:
obs_samples = pd.read_pickle('_!obs_samples!_.pickle')
obs_samples.head(2)

0    1    2    3    4    5    6    7    8    9    \
DS  product version tseries                                                     
NVD chrome  1       6          0    1    2    4    4    6    9  NaN  NaN  NaN   
                    7          0    1    2    4    4    6    9   13  NaN  NaN   

                            ...   165  166  167  168  169  170  171  172  173  \
DS  product version tseries ...                                                 
NVD chrome  1       6       ...   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                    7       ...   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

                             174  
DS  product version tseries       
NVD chrome  1       6        NaN  
                    7        NaN  

[2 rows x 175 columns]

In [7]:
def model_LN(t, paramlist):
    """ Linear (LN) """
    A, B = paramlist
    return A*t + B

def model_RE(t, paramlist):
    """Rescorla Exponential (RE)"""
    B_0, B_1 = paramlist
    return B_0*log(1+B_1*t)

modelname = 'model_RE'

In [24]:
def error(data, output):
    #here we have to drop the T0, because the chisquare can't handle the zero. 
    #However, this should probably get evaluated as part of the fit, so not sure what to do here...
    output = output[data.notnull()][1:] 
    data = data[data.notnull()][1:]
    chi2 = stats.chisquare(data.values, output)
    return chi2.statistic, chi2.pvalue

error([1,2], obs_samples.iloc[0], model_RE, conn)

In [ ]:
def run(paramlist, data, func, db_conn):
    output = [func(t, paramlist) for t in data.index]
    a = pd.DataFrame(data=[output], columns=data.index) 
    a['B0'] = paramlist[0] #don't like this much...
    a['B1'] = paramlist[1]
    a.set_index(['B0','B1'], inplace=True)
    a.to_sql(name=modelname, con=conn, if_exists='append', index_label=['B0','B1'])
    
    #return error(data, a.iloc[0])

run([1,2], obs_samples.iloc[0], model_RE, conn)

In [23]:
data = obs_samples.iloc[0]
results_df = pd.read_sql('select * from %s;'%modelname, conn, index_col=['B0','B1'])
# each row is a simulation result at a different set of parameters
results_df = results_df[~results_df.index.duplicated(keep='first')] #drop duplicate indices
error = results_df.apply(lambda output: error(data, output)[0], axis=1)
error

IndexingError: ('Unalignable boolean Series key provided', u'occurred at index (1, 2)')

In [27]:
error(obs_samples.iloc[0], results_df.iloc[0])

IndexingError: Unalignable boolean Series key provided

In [ ]:
%debug

> /Users/houghton/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py(1698)check_bool_indexer()
   1697         if mask.any():
-> 1698             raise IndexingError('Unalignable boolean Series key provided')
   1699 

ipdb> u
> /Users/houghton/anaconda/lib/python2.7/site-packages/pandas/core/series.py(595)__getitem__()
    594         if is_bool_indexer(key):
--> 595             key = check_bool_indexer(self.index, key)
    596 

ipdb> u
> <ipython-input-24-e4955e9765f5>(4)error()
      3     #However, this should probably get evaluated as part of the fit, so not sure what to do here...
----> 4     output = output[data.notnull()][1:]
      5     data = data[data.notnull()][1:]

ipdb> data
0       0
1       1
2       2
3       4
4       4
5       6
6       9
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23    NaN
24    NaN
25    NaN
26    NaN
27    NaN
28    NaN
2

In [ ]:
def fit(data, run_func, bounds, db_conn):
    results_df = pd.read_sql('select * from %s;'%modelname, conn, index_col=['B0','B1'])
    # each row is a simulation result at a different set of parameters
    results_df = results_df[~results_df.index.duplicated(keep='first')] #drop duplicate indices
    error = results_df.apply(lambda output: error(data, output), axis=1)
    

In [ ]:
def fit(row, error_func, guess, bounds, maxiter):
    """
    Parameters
    ----------
    error : function
    """
    data = row.dropna()
    
    def func(paramlist, data):
        # the error func also returns the 
        return error_func(paramlist, data)[0]
    
    res = scipy.optimize.minimize(fun=func, 
                                  x0=guess,
                                  method='L-BFGS-B',
                                  bounds=bounds,
                                  args=(data), 
                                  options={'maxiter':100})
    
    return pd.Series(error_func(res['x'],data), index=['chi2','pval'])

modified nelder-mead

in n-d space:

1. find best n+1 best points you know about 
2. calculate the `reflected` point - ie, the reflection of the lowest point across the centroid
3. evaluate the `reflected` point
4. if the `reflected` point is better than the worst point
 1. if the `reflected` point is better than the best point
   1. calculate the `expanded` point
   2. evaluate the `expanded` point
   3. return to beginning #not strict to nelder mead  
 2. else if the `reflected` point is not better than the best point, return to beginning
5. if the `reflected` point is worse than the worst point
 1. calculate the `contracted` point
 2. evalueate the `contracted` point
 3. if the `contracted` point is better than the worst point
   1. return to the beginning
 4. else if the `contracted` point is not better than the worst point


    expand the simplex: increase the distance by a constant factor in the same direction, try again
otherwise, 
    contract the simplex: decrease the distance of the worst point from the better two
    
    
This would be nice if the model was deterministic. with the stochastic return value, its hard to 

In [ ]:
modified nelder-mead




Lets make something up:
    
1. Find the lowest n points in the search space, 
2. calculate their maximal separation from each other $r$
3. fit a quadratic curve to all points within $m\cdot r$
4. find the minimum of the quadratic, and evaluate it
5. repeat




Really what we want is the set of parameters that are most likely to have been the ones that generated the observed data, or we want a maximum likelihood estimate of the parameters of the model, given the data. This is different from the best fit to the data. In truth, each different piece of software should dell us something about the parameters, across products, if they are in any way similar.

Perhaps the way to do it is to take the model as simplified by a single actor, and find an analytical solution for the expected behavior? Or take an average of several simulation results at one location, and compare them with the observed data? 